In [1]:
from bs4 import BeautifulSoup
from scrape import extract
from scrape import extract_players
from scrape import extract_year
from scrape import make_dists
from scrape import extract_age
from scrape import run_knn
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
import re

In [2]:
df2022 = extract_players("2022", "wr", "PPR")

In [3]:
names2022 = list(df2022["Name"].head(100))
dists2022 = make_dists(names2022, "2022")
dists2022.head(20)

justin-jefferson
tyreek-hill
davante-adams
stefon-diggs
ceedee-lamb
aj-brown
amonra-stbrown
jaylen-waddle
devonta-smith
amari-cooper
jamarr-chase
christian-kirk
tyler-lockett
terry-mclaurin
brandon-aiyuk
dk-metcalf
mike-evans
tee-higgins
chris-godwin
michael-pittman
garrett-wilson
jerry-jeudy
cooper-kupp
dj-moore
chris-olave
zay-jones
juju-smith-schuster
diontae-johnson
jakobi-meyers
adam-thielen
drake-london
mike-williams
curtis-samuel
tyler-boyd
allen-lazard
gabriel-davis
josh-palmer
deebo-samuel
donovan-peoples-jones
george-pickens
christian-watson
keenan-allen
courtland-sutton
marquise-brown
kj-osborn
mack-hollins
deandre-hopkins
parris-campbell
brandin-cooks
richie-james
jahan-dotson
darius-slayton
russell-gage
marquez-valdesscantling
isaiah-mckenzie
marvin-jones
kalif-raymond
devin-duvernay
robert-woods
chris-moore
noah-brown
greg-dortch
deandre-carter
alec-pierce
olamide-zaccheaus
michael-gallup
chase-claypool
josh-reynolds
demarcus-robinson
devante-parker
romeo-doubs
darnell-mo

,name,age,avg_rec,var_rec,avg_tgt,var_tgt,avg_yds,var_yds,avg_ypr,var_ypr,avg_lg,var_lg,avg_TD,var_TD,avg_rush,var_rush,avg_ryds,var_ryds,avg_rTD,var_rTD
0,justin-jefferson,23,14.222222,27.787998,20.444444,39.845845,201.000000,394.765894,13.527778,4.513659,32.111111,17.460714,0.888889,1.822358,0.444444,1.012270,2.666667,6.137318,0.111111,0.31427
1,tyreek-hill,28,13.222222,25.833990,18.888889,36.816193,190.000000,372.409721,14.566667,4.646743,35.500000,17.484120,0.777778,1.617802,0.777778,1.617802,3.555556,7.747560,0.111111,0.31427
2,davante-adams,29,11.111111,21.740615,20.000000,38.940125,168.444444,331.002723,14.777778,8.635171,31.166667,17.680026,1.555556,3.130890,0.333333,0.745356,-0.111111,1.369870,0.000000,0.00000
3,stefon-diggs,28,12.222222,23.889922,17.333333,33.867388,161.666667,315.834169,13.427778,2.323026,33.555556,13.030828,1.222222,2.484520,0.111111,0.314270,-0.333333,0.942809,0.000000,0.00000
4,ceedee-lamb,23,11.888889,23.184978,17.333333,33.745782,151.000000,294.678733,12.927778,3.189707,25.944444,7.940558,1.000000,2.054805,1.111111,2.282515,5.222222,11.792444,0.000000,0.00000
5,aj-brown,25,9.777778,19.083313,16.111111,31.377432,166.222222,325.377995,16.416667,4.818627,38.777778,19.543557,1.222222,2.506781,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
6,amonra-stbrown,22,12.470588,23.504987,17.176471,32.333256,136.588235,258.047889,10.582353,2.945832,24.294118,11.589877,0.705882,1.485804,1.058824,2.071390,11.176471,26.201112,0.000000,0.00000
7,jaylen-waddle,23,8.333333,16.323126,13.000000,25.458026,150.666667,295.347065,18.227778,7.109212,37.222222,22.594384,0.888889,1.852592,0.333333,0.745356,2.888889,6.462618,0.000000,0.00000
8,devonta-smith,23,10.555556,20.618822,15.111111,29.457859,132.888889,261.329696,11.711111,5.917759,23.888889,12.999525,0.777778,1.617802,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
9,amari-cooper,28,8.666667,16.960739,14.666667,28.641462,128.888889,252.307768,15.788889,7.381651,28.777778,15.526163,1.000000,2.054805,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [29]:
results2022 = run_knn(dists2022, 5)

TypeError: run_knn() takes 1 positional argument but 2 were given

In [ ]:
results2022[results2022["class"] == 4]

In [5]:
results2022[results2022["class"] == 3]

,name,class
39,george-pickens,3
40,christian-watson,3
54,isaiah-mckenzie,3
57,devin-duvernay,3
72,rashid-shaheed,3
77,mecole-hardman,3
94,laviska-shenault,3
95,jamal-agnew,3
97,tyquan-thornton,3


In [6]:
results2022[results2022['class'] == 2]

,name,class
32,curtis-samuel,2
37,deebo-samuel,2


In [7]:
results2022[results2022["class"] == 1]

,name,class
0,justin-jefferson,1
1,tyreek-hill,1
2,davante-adams,1
3,stefon-diggs,1
4,ceedee-lamb,1
5,aj-brown,1
6,amonra-stbrown,1
7,jaylen-waddle,1
8,devonta-smith,1
9,amari-cooper,1


In [8]:
results2022[results2022["class"] == 0]

,name,class
29,adam-thielen,0
33,tyler-boyd,0
36,josh-palmer,0
38,donovan-peoples-jones,0
42,courtland-sutton,0
44,kj-osborn,0
45,mack-hollins,0
47,parris-campbell,0
49,richie-james,0
50,jahan-dotson,0


2021 Data

In [9]:
df2021 = extract_players("2021", "wr", "PPR")
df2021.head()

,Name,RECEIVING_REC,RECEIVING_TGT,RECEIVING_YDS,RECEIVING_Y/R,RECEIVING_LG,RECEIVING_20+,RECEIVING_TD,RUSHING_ATT,RUSHING_YDS,RUSHING_TD,MISC_FL,MISC_G,MISC_FPTS,MISC_FPTS/G,MISC_ROST
Rank,,,,,,,,,,,,,,,,
1,cooper-kupp,145,191,1947,13.4,59,57,16,4,18,0,0,17,439.5,25.9,99.9%
2,davante-adams,123,169,1553,12.6,59,37,11,0,0,0,0,16,344.3,21.5,99.9%
3,deebo-samuel,77,121,1405,18.2,83,53,6,59,365,8,2,16,339.0,21.2,98.7%
4,justin-jefferson,108,167,1616,15.0,56,45,10,6,14,0,1,17,330.4,19.4,100.0%
5,jamarr-chase,81,128,1455,18.0,82,49,13,7,21,0,1,17,304.6,17.9,99.9%


In [10]:
names2021 = list(df2021["Name"].head(100))
dists2021 = make_dists(names2021, "2021")
dists2021.head(20)

cooper-kupp
davante-adams
deebo-samuel
justin-jefferson
jamarr-chase
tyreek-hill
stefon-diggs
diontae-johnson
mike-evans
hunter-renfrow
keenan-allen
mike-williams
jaylen-waddle
dk-metcalf
chris-godwin
tyler-lockett
michael-pittman
dj-moore
ceedee-lamb
brandin-cooks
amonra-stbrown
marquise-brown
darnell-mooney
tee-higgins
terry-mclaurin
christian-kirk
amari-cooper
adam-thielen
jakobi-meyers
devonta-smith
tyler-boyd
aj-brown
kendrick-bourne
marvin-jones
brandon-aiyuk
van-jefferson
chase-claypool
russell-gage
cole-beasley
kj-osborn
aj-green
tim-patrick
marquez-callaway
courtland-sutton
cedrick-wilson
deandre-hopkins
allen-lazard
mecole-hardman
robby-anderson
elijah-moore
robert-woods
jarvis-landry
kalif-raymond
emmanuel-sanders
odell-beckham
laviska-shenault
byron-pringle
gabriel-davis
antonio-brown
braxton-berrios
quez-watkins
deonte-harty
donovan-peoples-jones
jamison-crowder
bryan-edwards
rondale-moore
nick-westbrook
zay-jones
corey-davis
devante-parker
rashod-bateman
nelson-agholor
ja

,name,age,avg_rec,var_rec,avg_tgt,var_tgt,avg_yds,var_yds,avg_ypr,var_ypr,avg_lg,var_lg,avg_TD,var_TD,avg_rush,var_rush,avg_ryds,var_ryds,avg_rTD,var_rTD
0,cooper-kupp,28,16.111111,31.320722,21.222222,41.231805,216.333333,420.389898,13.755556,2.629580,38.555556,12.698304,1.777778,3.536407,0.444444,0.955814,2.000000,5.859465,0.000000,0.000000
1,davante-adams,28,14.470588,27.226971,19.882353,37.447818,182.705882,344.941387,12.600000,3.720215,33.705882,13.389042,1.294118,2.538288,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,deebo-samuel,25,9.058824,17.165186,14.235294,26.899660,165.294118,314.331179,17.952941,4.503217,42.941176,23.405556,0.705882,1.445671,6.941176,13.361360,42.941176,83.885405,0.941176,1.862023
3,justin-jefferson,22,12.000000,23.390406,18.555556,36.117435,179.555556,350.721355,15.072222,4.126312,32.777778,13.074808,1.111111,2.233306,0.666667,1.374369,1.555556,4.387130,0.000000,0.000000
4,jamarr-chase,21,9.000000,17.619434,14.222222,27.737971,161.666667,320.295523,16.483333,6.738385,38.500000,24.171264,1.444444,2.929206,0.777778,1.651785,2.333333,5.416026,0.000000,0.000000
5,tyreek-hill,27,12.333333,24.175286,17.666667,34.544657,137.666667,272.439392,10.311111,4.212269,26.388889,20.434554,1.000000,2.108185,1.000000,2.000000,10.666667,22.597935,0.000000,0.000000
6,stefon-diggs,27,11.444444,22.291558,18.222222,35.458200,136.111111,265.821763,12.677778,6.261927,27.777778,16.507761,1.111111,2.233306,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,diontae-johnson,25,12.588235,23.676891,19.882353,37.430535,136.588235,257.252007,11.994118,6.765003,29.705882,13.580976,0.941176,1.862023,0.588235,1.239489,6.235294,13.379217,0.000000,0.000000
8,mike-evans,27,8.705882,16.452301,13.411765,25.316953,121.764706,230.300195,14.611765,5.645704,28.529412,11.366192,1.647059,3.216523,0.117647,0.322190,1.176471,3.221897,0.000000,0.000000
9,hunter-renfrow,25,11.444444,22.333886,14.222222,27.709917,115.333333,225.825154,10.116667,3.164253,22.833333,12.266712,1.000000,2.027588,0.333333,0.745356,0.333333,1.699673,0.000000,0.000000


In [11]:
results2021 = run_knn(dists2021)

c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
results2021[results2021["class"] == 3]

,name,class
38,cole-beasley,3
50,robert-woods,3
51,jarvis-landry,3
59,braxton-berrios,3
63,jamison-crowder,3
65,rondale-moore,3
66,nick-westbrook,3
70,rashod-bateman,3
73,zach-pascal,3
75,josh-palmer,3


In [13]:
results2021[results2021["class"] == 2]

,name,class
2,deebo-samuel,2


In [14]:
results2021[results2021["class"] == 1]

,name,class
0,cooper-kupp,1
1,davante-adams,1
3,justin-jefferson,1
4,jamarr-chase,1
5,tyreek-hill,1
6,stefon-diggs,1
7,diontae-johnson,1
8,mike-evans,1
9,hunter-renfrow,1
10,keenan-allen,1


In [15]:
results2021[results2021["class"] == 0]

,name,class
30,tyler-boyd,0
32,kendrick-bourne,0
34,brandon-aiyuk,0
35,van-jefferson,0
36,chase-claypool,0
39,kj-osborn,0
40,aj-green,0
41,tim-patrick,0
42,marquez-callaway,0
43,courtland-sutton,0


2020 Data

In [16]:
df2020 = extract_players("2020", "wr", "PPR")

In [17]:
names2020 = list(df2020["Name"].head(100))
dists2020 = make_dists(names2020, "2020")
dists2020.head(20)

davante-adams
tyreek-hill
stefon-diggs
deandre-hopkins
calvin-ridley
justin-jefferson
dk-metcalf
tyler-lockett
allen-robinson
adam-thielen
mike-evans
aj-brown
robert-woods
keenan-allen
amari-cooper
juju-smith-schuster
brandin-cooks
marvin-jones
robby-anderson
terry-mclaurin
diontae-johnson
ceedee-lamb
chase-claypool
curtis-samuel
dj-moore
cooper-kupp
cole-beasley
tee-higgins
tyler-boyd
corey-davis
chris-godwin
will-fuller
jarvis-landry
nelson-agholor
brandon-aiyuk
marquise-brown
russell-gage
michael-gallup
jamison-crowder
devante-parker
emmanuel-sanders
ty-hilton
sterling-shepard
tim-patrick
jerry-jeudy
laviska-shenault
keelan-cole
mike-williams
dj-chark
darnell-mooney
christian-kirk
julio-jones
jakobi-meyers
darius-slayton
marquez-valdesscantling
gabriel-davis
zach-pascal
greg-ward
hunter-renfrow
kendrick-bourne
mecole-hardman
josh-reynolds
rashard-higgins
david-moore
antonio-brown
travis-fulgham
damiere-byrd
aj-green
anthony-miller
demarcus-robinson
danny-amendola
trequan-smith
scott

,name,age,avg_rec,var_rec,avg_tgt,var_tgt,avg_yds,var_yds,avg_ypr,var_ypr,avg_lg,var_lg,avg_TD,var_TD,avg_rush,var_rush,avg_ryds,var_ryds,avg_rTD,var_rTD
0,davante-adams,27,15.333333,26.784739,19.866667,34.669615,183.200000,322.094437,11.593333,3.264448,30.866667,15.722030,2.400000,4.270831,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,tyreek-hill,26,10.875000,19.852188,16.875000,30.750762,159.500000,293.082753,15.325000,6.308080,33.875000,19.541222,1.875000,3.479853,1.625000,3.059310,15.375000,29.891209,0.250000,0.559017
2,stefon-diggs,26,14.941176,28.078609,19.529412,36.705694,180.588235,340.435544,12.100000,3.672153,29.529412,15.499972,0.941176,1.924168,0.117647,0.322190,0.117647,0.322190,0.000000,0.000000
3,deandre-hopkins,28,13.529412,25.525345,18.823529,35.410592,165.529412,313.497184,13.188235,7.303495,30.647059,15.962976,0.705882,1.404393,0.117647,0.322190,0.117647,0.322190,0.000000,0.000000
4,calvin-ridley,25,11.250000,20.467963,17.875000,32.418889,171.750000,313.183552,14.656250,6.098357,31.250000,17.130018,1.125000,2.146946,0.625000,1.218349,0.125000,3.533324,0.000000,0.000000
5,justin-jefferson,21,10.352941,19.565700,14.705882,27.778038,164.705882,312.291298,15.958824,5.944455,33.764706,18.151420,0.823529,1.688553,0.117647,0.322190,0.235294,0.644379,0.000000,0.000000
6,dk-metcalf,22,9.764706,18.485382,15.176471,28.590898,153.294118,290.641636,15.935294,5.917068,33.882353,16.552527,1.176471,2.306903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,tyler-lockett,27,11.764706,22.312037,15.529412,29.396113,124.000000,235.879082,10.864706,2.965278,24.588235,11.956090,1.176471,2.431056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,allen-robinson,26,12.000000,22.624817,17.764706,33.463816,147.058824,276.919285,12.958824,3.454078,27.529412,7.554645,0.705882,1.445671,0.117647,0.322190,-0.117647,0.322190,0.000000,0.000000
9,adam-thielen,29,9.250000,16.891196,13.500000,24.568781,115.625000,211.644417,12.468750,3.629862,25.062500,13.315727,1.750000,3.250000,0.375000,0.780625,1.875000,4.013649,0.000000,0.000000


In [18]:
results2020 = run_knn(dists2020)

c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [19]:
results2020[results2020["class"] == 3]

,name,class
1,tyreek-hill,3
12,robert-woods,3
21,ceedee-lamb,3
22,chase-claypool,3
23,curtis-samuel,3
34,brandon-aiyuk,3
42,sterling-shepard,3
45,laviska-shenault,3
90,odell-beckham,3


In [20]:
results2020[results2020["class"] == 2]

,name,class
32,jarvis-landry,2
36,russell-gage,2
37,michael-gallup,2
39,devante-parker,2
40,emmanuel-sanders,2
41,ty-hilton,2
43,tim-patrick,2
46,keelan-cole,2
47,mike-williams-wr,2
49,darnell-mooney,2


In [25]:
results2020[results2020["class"] == 0]

,name,class
28,tyler-boyd,0
33,nelson-agholor,0
44,jerry-jeudy,0
50,christian-kirk,0
54,marquez-valdesscantling,0
55,gabriel-davis,0
60,mecole-hardman,0
63,david-moore,0
72,scott-miller,0
77,breshad-perriman,0


In [22]:
results2020[results2020["class"] == 0]

,name,class
28,tyler-boyd,0
33,nelson-agholor,0
44,jerry-jeudy,0
50,christian-kirk,0
54,marquez-valdesscantling,0
55,gabriel-davis,0
60,mecole-hardman,0
63,david-moore,0
72,scott-miller,0
77,breshad-perriman,0
